In [ ]:
pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import json

In [8]:
driver = webdriver.Chrome()
driver.get("https://www.gmarket.co.kr/")

### 로그인


In [11]:
# 이미 로그인되어 있는지 확인
if "로그인" not in driver.page_source:
    print("이미 로그인이 되어있음")

else:
    with open('gmarket_id.json', 'r') as json_file:
        data = json.load(json_file)
        username = data.get('username')
        password = data.get('password')


    login_link = driver.find_element(By.LINK_TEXT, "로그인")
    login_link.click()

    id_input = driver.find_element(By.ID, "typeMemberInputId")
    password_input = driver.find_element(By.ID, "typeMemberInputPassword")

    id_input.clear()
    id_input.send_keys(username)

    password_input.clear()
    password_input.send_keys(password)


    login_button = driver.find_element(By.ID, "btn_memberLogin")
    login_button.click()


### 상품 구매 및 환불 시나리오

In [7]:
# 검색 및 필터링 후 첫 상품 구매
search_box = driver.find_element(By.NAME, "keyword")
search_box.send_keys("생수")
search_box.send_keys(Keys.RETURN)
product_list1 = driver.find_elements(By.XPATH, "//div[@class='box__item-container']")
assert product_list1, "검색결과가 없음"

smile_delivery_checkbox = driver.find_element(By.XPATH, "//input[@value='스마일배송']")
smile_delivery_checkbox.click()

smile_product_list = driver.find_elements(By.XPATH, "//div[@class='box__item-container']")
assert smile_product_list, "스마일배송 상품 없음"

if smile_product_list:
    first_product = smile_product_list[0]
    product_link = first_product.find_element(By.TAG_NAME, 'a')
    product_url = product_link.get_attribute('href')
    driver.get(product_url)

select_button = driver.find_element(By.CSS_SELECTOR, "div.section_origin_area button.bt_select")
select_button.click()

buy_button = driver.find_element(By.ID, "coreInsOrderBtn")
buy_button.click()

wait = WebDriverWait(driver, 10)
time.sleep(5)

normal_payment_label = driver.find_element(By.CSS_SELECTOR, 'label[for="xo_id_radio_button_large_method_normal"].text__pay-title')
normal_payment_label.click()

virtual_account_radio = driver.find_element(By.XPATH, "//input[@id='pay_chk_VirtualAccount']")
driver.execute_script("arguments[0].click();", virtual_account_radio)
woori_bank = driver.find_element(By.XPATH, "//a[contains(., '우리은행')]")
woori_bank.click()

payment_button = driver.find_element(By.XPATH, "//button[contains(., '결제하기')]")
payment_button.click()

wait = WebDriverWait(driver, 15)
time.sleep(5)

order_history_button = driver.find_element(By.XPATH, "//button[@data-montelena-acode='200007340']")
order_history_button.click()


# 주문 취소
wait = WebDriverWait(driver, 15)
cancel_request_button = driver.find_element(By.XPATH, "//td[@class='b_table_right']//a[contains(text(), '취소신청')]")
cancel_request_button.click()

time.sleep(3)
iframe = driver.find_element(By.ID, 'popLayerIframe1')
driver.switch_to.frame(iframe)
cancel_complete_button = driver.find_element(By.LINK_TEXT, '취소 신청 완료')
cancel_complete_button.click()

time.sleep(3)
okay_button = driver.find_element(By.CLASS_NAME, 'link__confirm--active')
okay_button.click()
driver.switch_to.default_content()
